In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import os
import cv2
import numpy as np
from math import log10, sqrt

# Paths (adjust as needed)
input_dir = '/kaggle/input/imagenetsubsub'   # directory with original colored images
gray_dir = '/kaggle/working/gray_images/'     # directory to store grayscale images
colorized_dir = '/kaggle/working/colorized_images/' # directory to store colorized images

# Ensure output directories exist
os.makedirs(gray_dir, exist_ok=True)
os.makedirs(colorized_dir, exist_ok=True)

# Paths to model files (place them in a known location)
prototxt_path = '/kaggle/input/newmodelimagecolor/other/default/1/colorization_deploy_v2.prototxt'
caffemodel_path = '/kaggle/input/newmodelimagecolor/other/default/1/colorization_release_v2.caffemodel'
pts_in_hull_path = '/kaggle/input/newmodelimagecolor/other/default/1/pts_in_hull.npy'

# Load the cluster centers
pts_in_hull = np.load(pts_in_hull_path)
 
# Load the pre-trained model
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Populate the cluster centers as 1x1 convolution kernel
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts_in_hull.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype(np.float32)]
net.getLayer(conv8).blobs = [np.full([1,313], 2.606, np.float32)] # This is the default scale factor from the authors

def convert_to_grayscale_and_save(image_path, save_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Convert single-channel grayscale to 3-channel grayscale for consistency
    gray_3ch = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(save_path, gray_3ch)
    return img, gray_3ch

def colorize_image(gray_img):
    # Convert grayscale image (BGR) to Lab
    img_lab = cv2.cvtColor(gray_img, cv2.COLOR_BGR2LAB)
    L_channel = img_lab[:, :, 0].astype("float32") / 255.0

    # Create the input blob from L channel
    blob = cv2.dnn.blobFromImage(L_channel)  # shape (1,1,H,W)
    net.setInput(blob)

    # Forward pass through the network
    # Only get the 'class8_ab' output layer
    ab_dec = net.forward('class8_ab')[0, :, :, :]  # shape: (2, h', w')
    # Transpose to make it (h', w', 2)
    ab_dec = ab_dec.transpose((1, 2, 0))

    # Resize the predicted ab map to the size of the original image
    h, w, _ = gray_img.shape
    ab_dec = cv2.resize(ab_dec, (w, h))

    # Recombine L with ab
    L = L_channel * 100  # Lab L channel scale [0,100]
    a = ab_dec[:, :, 0] * 128
    b = ab_dec[:, :, 1] * 128

    color_lab = np.zeros((h, w, 3), dtype=np.float32)
    color_lab[:, :, 0] = L
    color_lab[:, :, 1] = a
    color_lab[:, :, 2] = b

    # Convert LAB back to BGR
    color_bgr = cv2.cvtColor(color_lab, cv2.COLOR_Lab2BGR)
    color_bgr = np.clip(color_bgr, 0, 255).astype("uint8")

    return color_bgr



def mse(img1, img2):
    # Mean Squared Error between two images
    err = np.mean((img1.astype("float") - img2.astype("float")) ** 2)
    return err

def psnr(img1, img2):
    # Peak Signal-to-Noise Ratio
    # First compute the MSE
    m = mse(img1, img2)
    if m == 0:
        return 100
    return 10 * log10((255**2) / m)

# Collect all images from the input directory
image_paths = glob.glob(os.path.join(input_dir, '*.*'))
 
psnr_values = []

for img_path in image_paths:
    filename = os.path.basename(img_path)
    gray_path = os.path.join(gray_dir, filename)
    colorized_path = os.path.join(colorized_dir, filename)
    
    # Convert to grayscale and save
    original, gray_img = convert_to_grayscale_and_save(img_path, gray_path)
    
    # Colorize using the pre-trained model
    colorized_img = colorize_image(gray_img)
    cv2.imwrite(colorized_path, colorized_img)
    
    # Calculate a metric (e.g., PSNR) to evaluate "accuracy"
    current_psnr = psnr(original, colorized_img)
    psnr_values.append(current_psnr)

# Compute average PSNR across all images
avg_psnr = np.mean(psnr_values)
print("Average PSNR across all colorized images:", avg_psnr)


In [ ]:
!pip install net

In [ ]:
import matplotlib.pyplot as plt

def visualize_images(original, grayscale, colorized):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Original")
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Grayscale")
    plt.imshow(cv2.cvtColor(grayscale, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Colorized")
    plt.imshow(cv2.cvtColor(colorized, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.show()


In [ ]:
import glob
import os
import cv2
import numpy as np
from math import log10, sqrt

# Input and output paths
input_dir = '/kaggle/input/imagenetsubsub/'
gray_dir = '/kaggle/working/gray_images/'
colorized_dir = '/kaggle/working/colorized_images/'

os.makedirs(gray_dir, exist_ok=True)
os.makedirs(colorized_dir, exist_ok=True)

# Paths to model files (adjust if placed elsewhere)
prototxt_path = '/kaggle/input/newmodelimagecolor/other/default/1/colorization_deploy_v2.prototxt'
caffemodel_path = '/kaggle/input/newmodelimagecolor/other/default/1/colorization_release_v2.caffemodel'
pts_in_hull_path = '/kaggle/input/newmodelimagecolor/other/default/1/pts_in_hull.npy'

# Constants based on the model's requirements
W_in = 224
H_in = 224

# Load cluster centers
pts_in_hull = np.load(pts_in_hull_path)

# Load the pre-trained model
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Populate cluster centers as layer blobs
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts_in_hull.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype(np.float32)]
net.getLayer(conv8).blobs = [np.full([1,313], 2.606, np.float32)]

# Attempt to use GPU (comment out if CUDA not available)
try:
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    print("Using CUDA backend and target for DNN.")
except:
    print("CUDA not available, using default backend.")

def convert_to_grayscale_and_save(image_path, save_path):
    img = cv2.imread(image_path)
    if img is None:
        return None, None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_3ch = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(save_path, gray_3ch)
    return img, gray_3ch

In [ ]:
def colorize_image(gray_img):
    # Convert BGR to Lab
    img_lab = cv2.cvtColor(gray_img, cv2.COLOR_BGR2Lab)
    img_l = img_lab[:, :, 0]  # Extract L channel

    # Resize L channel to model's input size and subtract 50 for mean-centering
    img_l_rs = cv2.resize(img_l, (W_in, H_in)).astype(np.float32) - 50.0
    print("Shape of img_l_rs:", img_l_rs.shape, "Data type:", img_l_rs.dtype)

    # Create blob and set as input to the network
    blob = cv2.dnn.blobFromImage(img_l_rs)
    net.setInput(blob)

    # Forward pass to predict 'ab' channels
    ab_dec = net.forward()[0, :, :, :]
    ab_dec = ab_dec.transpose((1, 2, 0))  # (H, W, 2)
    print("Shape of ab_dec:", ab_dec.shape, "Min:", ab_dec.min(), "Max:", ab_dec.max())

    # Normalize and clip predicted 'ab' channels to [-1, 1]
    ab_dec = np.clip(ab_dec, -1.0, 1.0)

    # Resize predicted 'ab' to original image size
    h, w = gray_img.shape[:2]
    ab_dec_us = cv2.resize(ab_dec, (w, h))
    print("Shape of ab_dec_us:", ab_dec_us.shape)

    # Reconstruct the Lab image
    lab_out = np.zeros((h, w, 3), dtype=np.float32)
    lab_out[:, :, 0] = img_l * (100.0 / 255.0)  # Scale L to [0, 100]
    lab_out[:, :, 1:] = ab_dec_us * 128  # Scale back 'ab' channels

    # Force valid range
    lab_out[:, :, 0] = np.clip(lab_out[:, :, 0], 0, 100)  # Ensure L-channel is in [0, 100]
    lab_out[:, :, 1:] = np.clip(lab_out[:, :, 1:], -128, 127)  # Ensure a/b channels are in [-128, 127]

    # Convert to uint8 for OpenCV compatibility
    lab_out_uint8 = np.clip(lab_out, 0, 255).astype(np.uint8)

    # Debug: Visualize the Lab channels
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("L-channel")
    plt.imshow(lab_out[:, :, 0], cmap='gray')
    plt.colorbar()

    plt.subplot(1, 3, 2)
    plt.title("a-channel")
    plt.imshow(lab_out[:, :, 1], cmap='jet')
    plt.colorbar()

    plt.subplot(1, 3, 3)
    plt.title("b-channel")
    plt.imshow(lab_out[:, :, 2], cmap='jet')
    plt.colorbar()

    plt.show()

    # Convert Lab back to BGR
    bgr_out = cv2.cvtColor(lab_out_uint8, cv2.COLOR_Lab2BGR)
    bgr_out = np.clip(bgr_out, 0, 255).astype(np.uint8)
    return bgr_out


In [ ]:
def mse(img1, img2):
    err = np.mean((img1.astype("float") - img2.astype("float")) ** 2)
    return err

def psnr(img1, img2):
    m = mse(img1, img2)
    if m == 0:
        return 100
    return 10 * log10((255**2) / m)

image_paths = glob.glob(os.path.join(input_dir, '*.*'))
image_paths = image_paths[:100]  # Limit to 100 images

psnr_values = []

for img_path in image_paths:
    filename = os.path.basename(img_path)
    gray_path = os.path.join(gray_dir, filename)
    colorized_path = os.path.join(colorized_dir, filename)

    original, gray_img = convert_to_grayscale_and_save(img_path, gray_path)
    
    # If reading the image failed, continue
    if original is None or gray_img is None:
        continue

    colorized_img = colorize_image(gray_img)
    cv2.imwrite(colorized_path, colorized_img)

    current_psnr = psnr(original, colorized_img)
    psnr_values.append(current_psnr)
    visualize_images(original, gray_img, colorized_img)


avg_psnr = np.mean(psnr_values) if psnr_values else 0
print("Average PSNR across all colorized images:", avg_psnr)
